In [1]:
import os
import torch
from tqdm import tqdm
from datasets import load_dataset
from torch.utils.data import DataLoader

from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType, PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, get_linear_schedule_with_warmup

# Setup

In [2]:
lr = 1e-2
num_epochs = 5
batch_size = 8
max_length = 128
text_column = "sentence"
label_column = "text_label"

device = "cuda:0"
model_name_or_path = "/home/wcs/Model/T5/t5-small/"
tokenizer_name_or_path = "/home/wcs/Model/T5/t5-small/"

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['http_proxy'] = "http://10.129.64.13:7890" 
os.environ['https_proxy'] = "http://10.129.64.13:7890" 

# Load Dataset

In [3]:
dataset = load_dataset("financial_phrasebank", "sentences_allagree")
dataset = dataset["train"].train_test_split(test_size=0.1)
dataset["validation"] = dataset["test"]
del dataset["test"]

classes = dataset["train"].features["label"].names
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1,
)

dataset["train"][0]

Map:   0%|          | 0/2037 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

{'sentence': 'The ship unloader is totally enclosed along the entire conveying line to the storage facilities .',
 'label': 1,
 'text_label': 'neutral'}

# Preprocess Dataset

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=2, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs

processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

Running tokenizer on dataset:   0%|          | 0/2037 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/227 [00:00<?, ? examples/s]

# Model

In [5]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, num_virtual_tokens=20)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 122,880 || all params: 60,629,504 || trainable%: 0.20267360260773368


# Optimizer

In [6]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

# Training

In [7]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████████████████████████████████████| 29/29 [00:00<00:00, 177.12it/s]


epoch=0: train_ppl=tensor(3.6653, device='cuda:0') train_epoch_loss=tensor(1.2989, device='cuda:0') eval_ppl=tensor(1.3109, device='cuda:0') eval_epoch_loss=tensor(0.2707, device='cuda:0')


100%|██████████████████████████████████████████| 29/29 [00:00<00:00, 180.05it/s]


epoch=1: train_ppl=tensor(1.4008, device='cuda:0') train_epoch_loss=tensor(0.3371, device='cuda:0') eval_ppl=tensor(1.2165, device='cuda:0') eval_epoch_loss=tensor(0.1960, device='cuda:0')


100%|██████████████████████████████████████████| 29/29 [00:00<00:00, 179.47it/s]


epoch=2: train_ppl=tensor(1.3123, device='cuda:0') train_epoch_loss=tensor(0.2718, device='cuda:0') eval_ppl=tensor(1.1830, device='cuda:0') eval_epoch_loss=tensor(0.1681, device='cuda:0')


100%|██████████████████████████████████████████| 29/29 [00:00<00:00, 179.53it/s]


epoch=3: train_ppl=tensor(1.2832, device='cuda:0') train_epoch_loss=tensor(0.2494, device='cuda:0') eval_ppl=tensor(1.1690, device='cuda:0') eval_epoch_loss=tensor(0.1562, device='cuda:0')


100%|██████████████████████████████████████████| 29/29 [00:00<00:00, 179.45it/s]

epoch=4: train_ppl=tensor(1.2636, device='cuda:0') train_epoch_loss=tensor(0.2340, device='cuda:0') eval_ppl=tensor(1.1625, device='cuda:0') eval_epoch_loss=tensor(0.1506, device='cuda:0')


# Evaluation

In [8]:
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100

print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['validation']['text_label'][:10]=}")

accuracy=89.42731277533039 % on the evaluation dataset
eval_preds[:10]=['positive', 'neutral', 'neutral', 'positive', 'neutral', 'negative', 'neutral', 'positive', 'neutral', 'positive']
dataset['validation']['text_label'][:10]=['positive', 'neutral', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'positive', 'neutral', 'positive']


# Save Model

In [9]:
peft_model_id = f"{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

# ckpt = f"{peft_model_id}/adapter_model.bin"
# !du -h $ckpt

/home/wcs/anaconda3/envs/torch/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /home/wcs/Model/T5/t5-small/ - will assume that the vocabulary was not modified.
  warnings.warn(


# Verification

In [10]:
peft_model_id = f"{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

inputs = tokenizer(
    "The Lithuanian beer market made up 14.41 million liters in January , a rise of 0.8 percent from the year-earlier figure , the Lithuanian Brewers ' Association reporting citing the results from its members .",
    return_tensors="pt",
)

model.to(device)

model.eval()
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['positive']
